In [ ]:
import pandas as pd 
import pysam
import os
import glob

In [ ]:
all_contig_anno = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230508refine_bg/tcr/M_0727PBMC1_NMc2_T3Nlib/03.assemble/M_0727PBMC1_NMc2_T3Nlib/outs/all_contig_annotations.csv")

In [ ]:
df_productive = all_contig_anno[all_contig_anno["productive"] == True]

In [ ]:
df_refine = df_productive[df_productive["is_cell"] == False]

In [ ]:
df_refine = df_refine.groupby("barcode").filter(lambda x: (len(x) > 4))

In [ ]:
df_refine

In [ ]:
# low confidence more than four productive contigs.

In [ ]:
df_sorted = df.sort_values(['barcode', 'chain', 'umis'], ascending=[False, False, False])

In [ ]:
df_sorted

In [ ]:
grouped = df_sorted.groupby(['barcode', 'chain'])
filtered_data = pd.DataFrame(columns=df_sorted.columns)  # 创建一个空的DataFrame来存储过滤后的数据
contig_id_set = set()  # 用于记录需要保留的contig_id

In [ ]:
        for group_name, group_data in grouped:
            if len(group_data) == 1:
                contig_id_set.add(group_data['contig_id'].values[0])  
            else:
                sorted_umis = group_data['umis'].values
                highest_umi = sorted_umis[0]
                second_highest_umi = sorted_umis[1]
                if highest_umi > second_highest_umi * 1.5:
                    contig_id_set.add(group_data.iloc[0]['contig_id'])  # 记录保留的contig_id

In [ ]:
for group_name, group_data in grouped:
    print(group_name)
    print(group_data)
    break

In [ ]:
# clonotypes

In [284]:
df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230508refine_bg/tcr/M_0727PBMC1_NMc2_T3Nlib/05.match/matched_contig_annotations.csv")

In [285]:
    df = df[df['productive'] == True]
    df['chain_cdr3aa'] = df[['chain', 'cdr3']].apply(':'.join, axis=1)
    df = df.rename(columns={"chain_cdr3aa":"cdr3s_aa", "raw_clonotype_id":"clonotype_id"})
    df = df.sort_values("clonotype_id", key=lambda x: x.str.lstrip("clonotype").astype(int))

In [286]:
    sort_method = {"TCR": True, "BCR": False}
    cdr3_aa_dict = df.groupby("clonotype_id")["cdr3s_aa"].apply(set).to_dict()
    cdr3_aa_dict = {key: ';'.join(sorted(list(value), reverse=sort_method["TCR"])) for key, value in cdr3_aa_dict.items()}
    count_dict = df.groupby("clonotype_id")["barcode"].nunique().to_dict()

In [287]:
    df["frequency"] = df["clonotype_id"].apply(lambda x: count_dict[x])
    df["cdr3s_aa"] = df["clonotype_id"].apply(lambda x: cdr3_aa_dict[x])
    df = df.drop_duplicates("clonotype_id")
    df = df[["clonotype_id", "cdr3s_aa", "frequency"]]

In [288]:
    sum_frequency = df['frequency'].sum()
    df['proportion'] = df['frequency'].apply(lambda x: x/sum_frequency)

In [289]:
df

,clonotype_id,cdr3s_aa,frequency,proportion
10794,clonotype1,TRB:CASSDLQGAAEQFF;TRA:CATDYQGGRALIF,44,0.006931
13533,clonotype2,TRB:CASSDLQGAAEQFF,6,0.000945
1353,clonotype3,TRB:CASSTWGGAQDTQYF;TRA:CAACMSNYNVLYF,11,0.001733
6100,clonotype4,TRB:CASRQSYNSPLYF;TRA:CALEPDTNTGKLTF,10,0.001575
10524,clonotype5,TRB:CASSLQGPNTEVFF;TRA:CAIVSGSWQLIF,10,0.001575
...,...,...,...,...
981,clonotype7270,TRB:CASGVRDWGIANTGQLYF;TRA:CALGAGTGGYKVVF,1,0.000158
8827,clonotype7272,TRB:CASSRNRDWGANTGQLYF;TRA:CAMREGEGNMGYKLTF;TR...,1,0.000158
8022,clonotype7273,TRB:CASSLGLGGPSSAETLYF;TRA:CAAMATGGNNKLTF,1,0.000158
13641,clonotype7274,TRB:CASGDAGLGGRRGAETLYF,1,0.000158


In [269]:
sorted(["TRA","TRB"], reverse=True)

['TRB', 'TRA']

In [270]:
sorted(["IGH","IGL","IGK"], reverse=False)

['IGH', 'IGK', 'IGL']

In [283]:
df

,clonotype_id,cdr3s_aa,frequency,proportion
10794,clonotype1,TRB:CASSDLQGAAEQFF;TRA:CATDYQGGRALIF,44,0.006931
13533,clonotype2,TRB:CASSDLQGAAEQFF,6,0.000945
1353,clonotype3,TRB:CASSTWGGAQDTQYF;TRA:CAACMSNYNVLYF,11,0.001733
6100,clonotype4,TRB:CASRQSYNSPLYF;TRA:CALEPDTNTGKLTF,10,0.001575
10524,clonotype5,TRB:CASSLQGPNTEVFF;TRA:CAIVSGSWQLIF,10,0.001575
...,...,...,...,...
981,clonotype7270,TRB:CASGVRDWGIANTGQLYF;TRA:CALGAGTGGYKVVF,1,0.000158
8827,clonotype7272,TRB:CASSRNRDWGANTGQLYF;TRA:CAMREGEGNMGYKLTF;TR...,1,0.000158
8022,clonotype7273,TRB:CASSLGLGGPSSAETLYF;TRA:CAAMATGGNNKLTF,1,0.000158
13641,clonotype7274,TRB:CASGDAGLGGRRGAETLYF,1,0.000158


In [22]:
import pandas as pd
import json
import pysam
import glob
import os
import io
import numpy as np
from collections import defaultdict

In [59]:
all_contig_anno = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230523refine/spleen_bcr/M_0727Pz2_B3Nlib/03.assemble/M_0727Pz2_B3Nlib/outs/all_contig_annotations.csv")
df_productive = all_contig_anno[all_contig_anno["productive"] == True]
df_refine = df_productive[df_productive["is_cell"] == False]
df_refine = df_refine.groupby("barcode").filter(lambda x: (len(x) > 4))

/SGRNJ/Public/Software/conda_env/chenjunjie/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,24,25,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [60]:
df_refine

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,...,fwr3_nt,cdr3,cdr3_nt,fwr4,fwr4_nt,reads,umis,raw_clonotype_id,raw_consensus_id,exact_subclonotype_id
51,AAACCTGAGAAGATTC-1,False,AAACCTGAGAAGATTC-1_contig_1,True,624,IGK,IGKV8-30,NaN,IGKJ1,IGKC,...,NaN,CHQYYSYPWTF,TGTCACCAATATTATAGCTATCCGTGGACGTTC,NaN,NaN,58455,8509,NaN,NaN,NaN
52,AAACCTGAGAAGATTC-1,False,AAACCTGAGAAGATTC-1_contig_2,True,480,IGH,IGHV8-12,NaN,IGHJ4,IGHG2B,...,NaN,CARRADYYYGLSMDYW,TGTGCTCGAAGAGCTGATTATTACTACGGCCTTTCTATGGACTACTGG,NaN,NaN,8,3,NaN,NaN,NaN
262,AAACCTGAGACTTGAA-1,False,AAACCTGAGACTTGAA-1_contig_1,False,619,IGK,IGKV3-7,NaN,IGKJ1,IGKC,...,NaN,CQHSWEIPWTF,TGTCAGCACAGTTGGGAGATTCCGTGGACGTTC,NaN,NaN,22840,1624,NaN,NaN,NaN
263,AAACCTGAGACTTGAA-1,False,AAACCTGAGACTTGAA-1_contig_2,False,577,IGH,IGHV9-3,IGHD2-4,IGHJ1,IGHG2B,...,NaN,CARWTMITRGWYFDVW,TGTGCAAGATGGACTATGATTACGAGAGGGTGGTACTTCGATGTCTGG,NaN,NaN,30159,1320,NaN,NaN,NaN
264,AAACCTGAGACTTGAA-1,False,AAACCTGAGACTTGAA-1_contig_3,False,619,IGK,IGKV3-12,NaN,IGKJ4,IGKC,...,NaN,CQHSRELPFTF,TGTCAGCACAGTAGGGAGCTTCCATTCACGTTC,NaN,NaN,19105,1488,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199432,CATCGAAAGAGTGAGA-1,False,CATCGAAAGAGTGAGA-1_contig_1,False,478,IGL,IGLV1,NaN,IGLJ3,IGLC3,...,NaN,CALWYSNHFIF,TGTGCTCTATGGTACAGCAACCATTTTATTTTC,NaN,NaN,5,2,NaN,NaN,NaN
199443,CATCGGGCACCCATTC-1,False,CATCGGGCACCCATTC-1_contig_1,False,444,IGH,IGHV1-72,IGHD4-1,IGHJ1,NaN,...,NaN,CASPNWDGGYFDVW,TGTGCAAGCCCGAACTGGGACGGTGGGTACTTCGATGTCTGG,NaN,NaN,5,1,NaN,NaN,NaN
199469,CATGGCGTCAACGCTA-1,False,CATGGCGTCAACGCTA-1_contig_1,False,426,IGK,IGKV15-103,NaN,IGKJ1,IGKC,...,NaN,CQQGQSYPWTF,TGTCAACAGGGTCAAAGTTATCCGTGGACTTTC,NaN,NaN,5,2,NaN,NaN,NaN
199483,CATTCGCAGTGTCCAT-1,False,CATTCGCAGTGTCCAT-1_contig_1,False,497,IGK,IGKV1-117,NaN,IGKJ2,IGKC,...,NaN,CFQGSHVPYTF,TGCTTTCAAGGTTCACATGTTCCGTACACGTTC,NaN,NaN,5,1,NaN,NaN,NaN


In [47]:
df = df_refine

In [48]:
seqtype ='BCR'

In [49]:
if seqtype == 'BCR':
    df['chain'] = df['chain'].replace(['IGL','IGK'],'IGK/L')

In [50]:
# df_sorted

In [51]:
        df_sorted = df.sort_values(['barcode', 'chain', 'umis'], ascending=[False, False, False])

        # 按barcode和chain进行groupby，保留满足条件的数据
        grouped = df_sorted.groupby(['barcode', 'chain'])
        filtered_data = pd.DataFrame(columns=df_sorted.columns)  # 创建一个空的DataFrame来存储过滤后的数据
        contig_id_set = set()  # 用于记录需要保留的contig_id

In [52]:
df_sorted

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,...,fwr3_nt,cdr3,cdr3_nt,fwr4,fwr4_nt,reads,umis,raw_clonotype_id,raw_consensus_id,exact_subclonotype_id
146978,AAGACCTGTAAATACG-1,False,AAGACCTGTAAATACG-1_contig_1,False,525,IGK/L,IGKV3-4,NaN,IGKJ1,IGKC,...,NaN,CQQTSEDPRTF,TGTCAGCAAACTAGTGAGGATCCTCGGACGTTC,NaN,NaN,54,8,NaN,NaN,NaN
146979,AAGACCTGTAAATACG-1,False,AAGACCTGTAAATACG-1_contig_2,False,471,IGH,IGHV5-16,NaN,IGHJ1,IGHM,...,NaN,CARDASYDGYWYFDVW,TGTGCAAGAGATGCGTCCTATGATGGTTACTGGTACTTCGATGTCTGG,NaN,NaN,25,4,NaN,NaN,NaN
146981,AAGACCTGTAAATACG-1,False,AAGACCTGTAAATACG-1_contig_4,False,470,IGH,IGHV1-55,IGHD1-1,IGHJ3,IGHM,...,NaN,CARGPYYYGSSYGFAYW,TGTGCAAGAGGACCTTATTACTACGGTAGTAGCTACGGGTTTGCTT...,NaN,NaN,7,2,NaN,NaN,NaN
146982,AAGACCTGTAAATACG-1,False,AAGACCTGTAAATACG-1_contig_5,False,545,IGH,IGHV1-55,IGHD1-1,IGHJ4,IGHD,...,NaN,CARSYYGSSYFYAMDYW,TGTGCAAGATCTTACTACGGTAGTAGCTACTTCTATGCTATGGACT...,NaN,NaN,6,2,NaN,NaN,NaN
146980,AAGACCTGTAAATACG-1,False,AAGACCTGTAAATACG-1_contig_3,False,524,IGH,IGHV7-3,NaN,IGHJ2,IGHM,...,NaN,CARGRTPFDYW,TGTGCAAGGGGTAGAACCCCGTTTGACTACTGG,NaN,NaN,35,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,AAACCTGAGCGATATA-1,False,AAACCTGAGCGATATA-1_contig_1,False,480,IGK/L,IGKV12-44,NaN,IGKJ1,IGKC,...,NaN,CQHHYGTPWTF,TGTCAACATCATTATGGTACTCCGTGGACGTTC,NaN,NaN,2436,202,NaN,NaN,NaN
943,AAACCTGAGCGATATA-1,False,AAACCTGAGCGATATA-1_contig_5,False,460,IGH,IGHV1-5,IGHD4-1,IGHJ2,IGHA,...,NaN,CNWDFDYW,TGTAATTGGGACTTTGACTACTGG,NaN,NaN,530,273,NaN,NaN,NaN
941,AAACCTGAGCGATATA-1,False,AAACCTGAGCGATATA-1_contig_3,False,537,IGH,IGHV8-12,NaN,IGHJ4,IGHG2B,...,NaN,CARRADYYYGLSMDYW,TGTGCTCGAAGAGCTGATTATTACTACGGCCTTTCTATGGACTACTGG,NaN,NaN,574,206,NaN,NaN,NaN
944,AAACCTGAGCGATATA-1,False,AAACCTGAGCGATATA-1_contig_6,False,471,IGH,IGHV5-16,NaN,IGHJ1,IGHM,...,NaN,CARDASYDGYWYFDVW,TGTGCAAGAGATGCGTCCTATGATGGTTACTGGTACTTCGATGTCTGG,NaN,NaN,138,57,NaN,NaN,NaN


In [53]:
        for _group_name, group_data in grouped:
            if len(group_data) == 1:
                contig_id_set.add(group_data['contig_id'].values[0])  
            else:
                sorted_umis = group_data['umis'].values
                highest_umi = sorted_umis[0]
                second_highest_umi = sorted_umis[1]
                if highest_umi >= second_highest_umi * 1.5:
                    contig_id_set.add(group_data.iloc[0]['contig_id'])  # 记录保留的contig_id

        filtered_data = df[df['contig_id'].isin(contig_id_set)]

In [ ]:
count = 0
for _group_name, group_data in grouped:
    count += 1
    if count <= 10:
        print(_group_name)
        print(group_data)
        print("\n==================")

In [54]:
filtered_data

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,...,fwr3_nt,cdr3,cdr3_nt,fwr4,fwr4_nt,reads,umis,raw_clonotype_id,raw_consensus_id,exact_subclonotype_id
1086,AAACCTGAGCGTAATA-1,False,AAACCTGAGCGTAATA-1_contig_1,False,457,IGH,IGHV1-53,NaN,IGHJ3,IGHM,...,NaN,CARGGERFAYW,TGTGCAAGAGGGGGCGAGAGGTTTGCTTACTGG,NaN,NaN,817,366,NaN,NaN,NaN
1371,AAACCTGAGCTAGGCA-1,False,AAACCTGAGCTAGGCA-1_contig_7,False,500,IGK/L,IGLV1,NaN,IGLJ1,IGLC1,...,NaN,CALWYSNLWVF,TGTGCTCTATGGTACAGCAACCTTTGGGTGTTC,NaN,NaN,558,242,NaN,NaN,NaN
1402,AAACCTGAGCTAGTGG-1,False,AAACCTGAGCTAGTGG-1_contig_5,False,434,IGK/L,IGKV3-4,NaN,IGKJ1,NaN,...,NaN,CQQSNEDPRTF,TGTCAGCAAAGTAATGAGGATCCTCGGACGTTC,NaN,NaN,630,299,NaN,NaN,NaN
1470,AAACCTGAGCTCCCAG-1,False,AAACCTGAGCTCCCAG-1_contig_2,False,476,IGH,IGHV1-5,IGHD4-1,IGHJ2,IGHA,...,NaN,CNWDFDYW,TGTAATTGGGACTTTGACTACTGG,NaN,NaN,419,225,NaN,NaN,NaN
1610,AAACCTGAGCTGGAAC-1,False,AAACCTGAGCTGGAAC-1_contig_2,False,493,IGK/L,IGKV3-4,NaN,IGKJ1,IGKC,...,NaN,CQQSNEDPRTF,TGTCAGCAAAGTAATGAGGATCCTCGGACGTTC,NaN,NaN,737,336,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136957,AACTCTTGTTCCGTCT-1,False,AACTCTTGTTCCGTCT-1_contig_2,False,687,IGK/L,IGKV1-122,NaN,IGKJ2,IGKC,...,NaN,CLQVTHVPYTF,TGCCTCCAAGTTACACATGTCCCGTACACGTTC,NaN,NaN,52,23,NaN,NaN,NaN
142006,AACTGGTTCGCCCTTA-1,False,AACTGGTTCGCCCTTA-1_contig_1,False,549,IGH,IGHV1-72,NaN,IGHJ4,IGHM,...,NaN,CARSMDYW,TGTGCAAGATCCATGGACTACTGG,NaN,NaN,83,8,NaN,NaN,NaN
142009,AACTGGTTCGCCCTTA-1,False,AACTGGTTCGCCCTTA-1_contig_4,False,488,IGK/L,IGKV6-25,NaN,IGKJ5,IGKC,...,NaN,CQQHYSTPLTF,TGTCAGCAACATTATAGCACTCCGCTCACGTTC,NaN,NaN,55,5,NaN,NaN,NaN
146978,AAGACCTGTAAATACG-1,False,AAGACCTGTAAATACG-1_contig_1,False,525,IGK/L,IGKV3-4,NaN,IGKJ1,IGKC,...,NaN,CQQTSEDPRTF,TGTCAGCAAACTAGTGAGGATCCTCGGACGTTC,NaN,NaN,54,8,NaN,NaN,NaN


In [55]:
        grouped_barcode = filtered_data.groupby('barcode')
        final_filtered_data = pd.DataFrame(columns=filtered_data.columns)
        contig_id_set = set()

        for _group_name, group_data in grouped_barcode:
            if len(group_data['chain'].unique()) == 2:
                if seqtype == 'BCR' and 'IGH' not in group_data['chain'].unique():
                    continue
                contig_id_set.update(set(group_data['contig_id']))  # 记录保留的contig_id

In [56]:
# contig_id_set

In [57]:
        # 根据contig_id确定需要保留的行
        final_filtered_data = filtered_data[filtered_data['contig_id'].isin(contig_id_set)]
        final_filtered_data = final_filtered_data.sort_values(['barcode'])

In [58]:
final_filtered_data

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,...,fwr3_nt,cdr3,cdr3_nt,fwr4,fwr4_nt,reads,umis,raw_clonotype_id,raw_consensus_id,exact_subclonotype_id
1826,AAACCTGAGGACGAAA-1,False,AAACCTGAGGACGAAA-1_contig_2,False,485,IGK/L,IGKV12-46,NaN,IGKJ5,IGKC,...,NaN,CQHFWGTPLTF,TGTCAACATTTTTGGGGTACTCCTCTCACGTTC,NaN,NaN,1623,120,NaN,NaN,NaN
1831,AAACCTGAGGACGAAA-1,False,AAACCTGAGGACGAAA-1_contig_7,False,613,IGH,IGHV1-80,NaN,IGHJ3,IGHM,...,NaN,CARDGNYGFAYW,TGTGCAAGAGATGGTAACTACGGGTTTGCTTACTGG,NaN,NaN,629,257,NaN,NaN,NaN
2500,AAACCTGAGGGAACGG-1,False,AAACCTGAGGGAACGG-1_contig_2,False,476,IGH,IGHV1-5,IGHD4-1,IGHJ2,IGHA,...,NaN,CNWDFDYW,TGTAATTGGGACTTTGACTACTGG,NaN,NaN,213,134,NaN,NaN,NaN
2503,AAACCTGAGGGAACGG-1,False,AAACCTGAGGGAACGG-1_contig_5,False,495,IGK/L,IGKV3-4,NaN,IGKJ1,IGKC,...,NaN,CQQSNEDPRTF,TGTCAGCAAAGTAATGAGGATCCTCGGACGTTC,NaN,NaN,657,340,NaN,NaN,NaN
2577,AAACCTGAGGGATGGG-1,False,AAACCTGAGGGATGGG-1_contig_1,False,441,IGH,IGHV1-81,NaN,IGHJ1,IGHM,...,NaN,CAGYYGSSWYFDVW,TGTGCCGGTTACTACGGTAGTAGCTGGTACTTCGATGTCTGG,NaN,NaN,320,90,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136744,AACTCTTGTGGTTTCA-1,False,AACTCTTGTGGTTTCA-1_contig_5,False,453,IGH,IGHV8-12,NaN,IGHJ1,IGHM,...,NaN,CARRRGLRWFFDVW,TGTGCTCGAAGGAGGGGACTACGGTGGTTCTTCGATGTCTGG,NaN,NaN,40,21,NaN,NaN,NaN
142006,AACTGGTTCGCCCTTA-1,False,AACTGGTTCGCCCTTA-1_contig_1,False,549,IGH,IGHV1-72,NaN,IGHJ4,IGHM,...,NaN,CARSMDYW,TGTGCAAGATCCATGGACTACTGG,NaN,NaN,83,8,NaN,NaN,NaN
142009,AACTGGTTCGCCCTTA-1,False,AACTGGTTCGCCCTTA-1_contig_4,False,488,IGK/L,IGKV6-25,NaN,IGKJ5,IGKC,...,NaN,CQQHYSTPLTF,TGTCAGCAACATTATAGCACTCCGCTCACGTTC,NaN,NaN,55,5,NaN,NaN,NaN
146978,AAGACCTGTAAATACG-1,False,AAGACCTGTAAATACG-1_contig_1,False,525,IGK/L,IGKV3-4,NaN,IGKJ1,IGKC,...,NaN,CQQTSEDPRTF,TGTCAGCAAACTAGTGAGGATCCTCGGACGTTC,NaN,NaN,54,8,NaN,NaN,NaN
